# 0. 기본 셋팅

In [3]:
####################### 모듈 불러오기 
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib as mpl  # 기본 설정 만지는 용도
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도

####################### 불필요한 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

####################### 여기서 Matplotlib 코드를 사용하여 그래프 생성 및 표시
%matplotlib inline  

####################### 한글폰트
#font_family = plt.rcParams['font.family']#메모리 상에 현재 설정되어 있는 폰트 패밀리 이름
#plt.rc('font', family='NanumSquare') #나눔 글꼴로 설정(이미 시스템에 설치되어있는경우 알아서 찾아다줌)
plt.rc('font', family='AppleGothic') #맥 
plt.rcParams['axes.unicode_minus'] = False #부호

####################### 데이터 불러오기
df = pd.read_csv('./data/서울특별시교육청남산도서관 장서 대출목록 (2023년 09월).csv',encoding='euc_kr')
df.head(2)
# ISBN : 도서를 식별하기 위한 국제 표준 번호 (일반적으로 각 도서에 고유한 번호) 
# 세트 ISBN : ISBN과 유사하지만, 보통 여러 도서를 묶어서 판매할 때 사용

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자,Unnamed: 13
0,1,이효석문학상 수상작품집:애도의 방식,안보윤 [외]지음,북다,2023,9791170610311,NaN,0,24,813.608,1,0,2023-09-25,NaN
1,2,인공지능을 위한 머신러닝과 딥러닝 with 파이썬,"김현정,유상현 (지은이)",길벗캠퍼스,2023,9791140706112,NaN,NaN,NaN,NaN,1,0,2023-09-25,NaN


In [4]:
df.info()

# ISBN : 도서를 식별하기 위한 국제 표준 번호 (일반적으로 각 도서에 고유한 번호) 
# 세트 ISBN : ISBN과 유사하지만, 보통 여러 도서를 묶어서 판매할 때 사용

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433903 entries, 0 to 433902
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   번호           433903 non-null  int64  
 1   도서명          433503 non-null  object 
 2   저자           433696 non-null  object 
 3   출판사          428405 non-null  object 
 4   발행년도         433860 non-null  object 
 5   ISBN         433903 non-null  object 
 6   세트 ISBN      26487 non-null   float64
 7   부가기호         335262 non-null  object 
 8   권            69777 non-null   object 
 9   주제분류번호       416756 non-null  object 
 10  도서권수         433903 non-null  int64  
 11  대출건수         433903 non-null  int64  
 12  등록일자         433903 non-null  object 
 13  Unnamed: 13  0 non-null       float64
dtypes: float64(2), int64(3), object(9)
memory usage: 46.3+ MB


# 1. 공공 API 불러오기 자동화

목표 : 20대가 좋아하는 도서 200개 목록 api 통해서 자동화 불러오기

In [22]:
####################### API 자동화 

# 지난 한 달간 "20대에서 인기 많은 대출 도서" API 신청 
# 도서관 정보나루 

#https://www.data4library.kr/apiUtilization# 참조 
# format : JSON 
# startDt : 검색 시작 일자, 2023년 8월 1일로 설정 
# ednDt : 검색 종료 일자, 2023년 8월 31일로 설정 
# age : 연령대 , 20대 로 설정 
# authKey : 인증키 발급 후 설정 

####################### API 신청후 '승인완료'되어야 사용 가능 

# 현재 승인신청중으로, 담당자 확인 있어야 사용가능 --> 승인신청 완료 
url = "http://data4library.kr/api/loanItemSrch?format=json&startDt=2023-08-01&endDt=2023-08-31&gender=0&from_age=6&to_age=20&region=11&addCode=0&kdc=8&authKey=f8c4d1efe4b15a61f4eef15cfd80369eff55e12f10763ad35ccc666504d516da"

####################### API 자동화 호출 
#!pip install requests
import requests

####################### URL을 requests.get()함수에 전달 
# 위의 url은 HTTP GET 방식으로 파라미터 값 전달하기 때문입니다. 

r = requests.get(url)
data = r.json()

In [23]:
# print(data)    # 너무 많이 출력되기때문에 일부만 캡쳐 

![](https://velog.velcdn.com/images/beckhye/post/f2024351-c656-48eb-8f04-57a75ee246f1/image.png)

In [24]:
# data    # 너무 많이 출력되기때문에 일부만 캡쳐 

![](https://velog.velcdn.com/images/beckhye/post/9834f23a-0085-41f7-bbcd-a09a56c651f0/image.png) <br><br>
.... <br><br>
![](https://velog.velcdn.com/images/beckhye/post/910aa933-ffb8-4b24-905e-ea8691dd13e2/image.png)

In [27]:
data.shape()         # json형태이기때문에 error 

AttributeError: 'dict' object has no attribute 'shape'

In [34]:
# 좀 더 편하게 보기 위하여 docs키에 매핑된 딕셔너리 추출한 후 리스트에 추가하는 반복문
books = []
for d in data['response']['docs']: 
    books.append(d['doc'])


# 위의 코드와 동일한 방식임
# 리스트 내포 List Comprehension 방식 코드 
books = [d['doc'] for d in data ['response']['docs']]

In [30]:
# book    # 너무 많이 출력되기때문에 일부만 캡쳐 

![](https://velog.velcdn.com/images/beckhye/post/f9695660-d171-4832-b710-94d44635ee22/image.png)<br><br>
...<br><br>
![](https://velog.velcdn.com/images/beckhye/post/076c3d01-e381-4053-8a98-00293065ceb5/image.png)

In [36]:
# 데이터프레임화 
books_df = pd.DataFrame(books)
books_df.head()

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,class_nm,loan_count,bookImageURL,bookDtlUrl
0,1,1,아몬드손원평 장편소설,손원평,창비,2017,9788936434267,03810,,813.7,문학 > 한국문학 > 소설,33,http://image.aladin.co.kr/product/16839/4/cove...,https://data4library.kr/bookV?seq=3423096
1,2,2,체리새우 :황영미 장편소설,지은이: 황영미,문학동네,2019,9788954654753,03810,,813.7,문학 > 한국문학 > 소설,32,https://image.aladin.co.kr/product/17970/0/cov...,https://data4library.kr/bookV?seq=5359157
2,3,3,불편한 편의점 :김호연 장편소설,지은이: 김호연,나무옆의자,2021,9791161571188,03810,,813.7,문학 > 한국문학 > 소설,29,https://image.aladin.co.kr/product/26942/84/co...,https://data4library.kr/bookV?seq=5962217
3,4,4,있으려나 서점,요시타케 신스케 지음 ;고향옥 옮김,김영사,2018,9788934981237,03830,,838,문학 > 일본문학 > 르포르타주 및 기타,25,https://image.aladin.co.kr/product/15618/36/co...,https://data4library.kr/bookV?seq=3824131
4,5,4,나미야 잡화점의 기적 :히가시노 게이고 장편소설,지은이: 히가시노 게이고 ;옮긴이: 양윤옥,현대문학,2012,9788972756194,03830,,833.6,문학 > 일본문학 > 소설,25,http://image.aladin.co.kr/product/15848/6/cove...,https://data4library.kr/bookV?seq=1902173


In [37]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   no                200 non-null    int64 
 1   ranking           200 non-null    object
 2   bookname          200 non-null    object
 3   authors           200 non-null    object
 4   publisher         200 non-null    object
 5   publication_year  200 non-null    object
 6   isbn13            200 non-null    object
 7   addition_symbol   200 non-null    object
 8   vol               200 non-null    object
 9   class_no          200 non-null    object
 10  class_nm          200 non-null    object
 11  loan_count        200 non-null    object
 12  bookImageURL      200 non-null    object
 13  bookDtlUrl        200 non-null    object
dtypes: int64(1), object(13)
memory usage: 22.0+ KB


In [38]:
books_df.shape

(200, 14)

지금 만든 프로그램을 정기적으로 실행하면 보고 자료 API를 호출할 수 있습니다.

In [39]:
# JSON파일로 변환하여 저장 
books_df.to_json('20s_best_book.json')

# 2. 웹 크롤링

목표 : 